In [ ]:
import numpy as np
import networkx as nx

nodes = [1, 2, 3, 4, 5]
edges = {
    (1, 2): 2, (1, 4): 5, (1, 5): 2,
    (2, 3): 1, (2, 4): 1, (2, 5): 1,
    (3, 4): 100, (4, 5): 1
}

G = nx.Graph()
for (i, j), cost in edges.items():
    G.add_edge(i, j, weight=cost)

nodes_excluding_1 = [n for n in nodes if n != 1]

In [ ]:
import scipy.optimize

# init Lagrange multipliers (node penalties)=0
node_penalties = {node: 0 for node in nodes}

#find 1-tree cost for given lambda
def compute_lagrangian_1_tree(penalties):
    # adj edge weights w/ penalties
    adjusted_graph = nx.Graph()
    for (u, v, data) in G.edges(data=True):
        adjusted_weight = data['weight'] + penalties[u-1] + penalties[v-1]s
        adjusted_graph.add_edge(u, v, weight=adjusted_weight)

    # find mst for N-{1}
    subgraph = adjusted_graph.subgraph(nodes_excluding_1)
    mst = nx.minimum_spanning_tree(subgraph, weight='weight')

    #add two edges to node 1
    edges_connected_to_1 = sorted([(1, j, adjusted_graph[1][j]['weight']) for j in G[1]], key=lambda x: x[2])
    one_tree_edges = list(mst.edges(data=True)) + [
        (edges_connected_to_1[0][0], edges_connected_to_1[0][1], {'weight': edges_connected_to_1[0][2]}),
        (edges_connected_to_1[1][0], edges_connected_to_1[1][1], {'weight': edges_connected_to_1[1][2]})
    ]

    #total adjusted 1-tree cost
    one_tree_cost = sum(edge[2]['weight'] for edge in one_tree_edges)

    # add neg penalty effects to get true cost
    total_penalty_adjustment = 2 * sum(penalties)  # each node will appears in two edges
    true_cost = one_tree_cost - total_penalty_adjustment

    return -true_cost  #negate for minimization

#optimize lambdas using gradient-free optimization
result = scipy.optimize.minimize(
    fun=compute_lagrangian_1_tree,
    x0=np.zeros(len(nodes)),
    method='Nelder-Mead'
)

#run
optimal_penalties = result.x
best_held_karp_bound = -compute_lagrangian_1_tree(optimal_penalties)

best_held_karp_bound


65.83394932481563